<a href="https://colab.research.google.com/github/SanshGoel/Mancala-AI/blob/main/Mancala_Project_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import copy
import math
import time
import random
from tqdm import tqdm

In [ ]:
class Mancala:
    def __init__(self, pits_per_player=6, stones_per_pit=4):
        # Initialize game parameters: number of pits per player and stones per pit
        self.pits_per_player = pits_per_player
        self.total_board_size = (pits_per_player + 1) * 2  # Total board size including Mancalas
        self.stones_per_pit = stones_per_pit
        self.reset_game()  # Set up the initial board state

    def reset_game(self):
        """Reset the game state without creating a new object."""
        # Initialize the board with stones and empty Mancalas
        self.board = [self.stones_per_pit] * self.total_board_size
        self.current_player = 1  # Player 1 starts the game
        self.board[self.pits_per_player] = 0  # Player 1's Mancala
        self.board[-1] = 0  # Player 2's Mancala

        # Define indices for Player 1 and Player 2's pits and Mancalas
        self.p1_pits_index = [0, self.pits_per_player - 1]
        self.p1_mancala_index = self.pits_per_player
        self.p2_pits_index = [self.pits_per_player + 1, self.total_board_size - 2]
        self.p2_mancala_index = self.total_board_size - 1

    def clone(self):
        """Create a lightweight copy of the game state."""
        new_game = Mancala(self.pits_per_player, self.stones_per_pit)
        new_game.board = self.board.copy()  # Copy the board state
        new_game.current_player = self.current_player  # Copy the current player
        return new_game

    def display_board(self):
        # Separate board elements for easier visualization
        player_1_pits = self.board[self.p1_pits_index[0]: self.p1_pits_index[1] + 1]
        player_1_mancala = self.board[self.p1_mancala_index]
        player_2_pits = self.board[self.p2_pits_index[0]: self.p2_pits_index[1] + 1]
        player_2_mancala = self.board[self.p2_mancala_index]

        # Print the board layout for debugging and gameplay
        print('P1               P2')
        print('     ____{}____     '.format(player_2_mancala))
        for i in range(self.pits_per_player):
            if i == self.pits_per_player - 1:
                print('{} -> |_{}_|_{}_| <- {}'.format(i + 1, player_1_pits[i], player_2_pits[-(i + 1)],
                                                       self.pits_per_player - i))
            else:
                print('{} -> | {} | {} | <- {}'.format(i + 1, player_1_pits[i], player_2_pits[-(i + 1)],
                                                      self.pits_per_player - i))

        print('         {}         '.format(player_1_mancala))
        turn = 'P1' if self.current_player == 1 else 'P2'
        print('Turn: ' + turn)

    def random_move_generator(self):
        """Generate random valid moves with non-empty pits."""
        # Determine valid moves based on the current player
        if self.current_player == 1:
            valid_moves = [i for i in range(0, self.pits_per_player) if self.board[i] > 0]
        else:
            valid_moves = [i for i in range(self.pits_per_player + 1, self.total_board_size - 1) if self.board[i] > 0]

        # Return a random valid move or -1 if no moves are possible
        return random.choice(valid_moves) + 1 if valid_moves else -1

    def valid_move(self, pit):
        """Check if the chosen pit is a valid move."""
        pit_index = pit - 1  # Convert 1-based index to 0-based
        current_player_pits_range = self.p1_pits_index if self.current_player == 1 else self.p2_pits_index

        # A move is valid if the pit belongs to the current player and is non-empty
        return (current_player_pits_range[0] <= pit_index <= current_player_pits_range[1]) and (self.board[pit_index] > 0)

    def play(self, pit, cur_player, debug=False):
        """Perform a move in the game."""
        self.current_player = cur_player  # Set the current player
        if pit <= 0:
            return self.board  # Return if the pit is invalid

        if debug:
            print(f'Player {self.current_player} chose pit: {pit}\n')

        if not self.valid_move(pit):
            if debug:
                print("Invalid move\n")
            return self.board  # Return if the move is invalid

        if self.winning_eval():
            if debug:
                print('Game Over')
            return self.board  # Return if the game is over

        pit_index = pit - 1  # Convert to 0-based index
        stones_to_distribute = self.board[pit_index]  # Number of stones in the chosen pit
        self.board[pit_index] = 0  # Empty the chosen pit

        # Distribute stones one by one into subsequent pits
        while stones_to_distribute > 0:
            pit_index = (pit_index + 1) % self.total_board_size
            # Skip the opponent's Mancala
            if (self.current_player == 1 and pit_index == self.p2_mancala_index) or \
               (self.current_player == 2 and pit_index == self.p1_mancala_index):
                continue
            self.board[pit_index] += 1
            stones_to_distribute -= 1

        # Check for capture rule: last stone lands in an empty pit on the current player's side
        if ((self.current_player == 1 and self.p1_pits_index[0] <= pit_index <= self.p1_pits_index[1] and self.board[pit_index] == 1) or
            (self.current_player == 2 and self.p2_pits_index[0] <= pit_index <= self.p2_pits_index[1] and self.board[pit_index] == 1)):
            opposite_pit_index = self.total_board_size - pit_index - 2  # Opposite pit index
            if self.board[opposite_pit_index] > 0:
                mancala_index = self.p1_mancala_index if self.current_player == 1 else self.p2_mancala_index
                self.board[mancala_index] += self.board[opposite_pit_index] + 1  # Capture stones
                self.board[opposite_pit_index] = 0
                self.board[pit_index] = 0

        # Switch to the other player
        self.current_player = 3 - self.current_player
        return self.board

    def winning_eval(self):
        """Evaluate if the game has reached a winning state."""
        # Sum the stones in each player's pits
        p1_pits_sum = sum(self.board[self.p1_pits_index[0]: self.p1_pits_index[1] + 1])
        p2_pits_sum = sum(self.board[self.p2_pits_index[0]: self.p2_pits_index[1] + 1])

        # Check if all pits on one side are empty
        if p1_pits_sum == 0 or p2_pits_sum == 0:
            if self.board[self.p1_mancala_index] > self.board[self.p2_mancala_index]:
                return "Player 1"  # Player 1 wins
            elif self.board[self.p1_mancala_index] < self.board[self.p2_mancala_index]:
                return "Player 2"  # Player 2 wins
            else:
                return "Tie"  # It's a tie
        return None  # Game is not over

    def match_analysis(self, games=100, debug=False, p1_type='random', p2_type='alpha-beta', p1_depth=5, p2_depth=5):
        """Analyze match performance between different player types."""
        p1_wins = p2_wins = ties = total_moves = 0

        progress_bar = tqdm(total=games)  # Progress bar for tracking games

        # Initialize AI players with specified depths
        ai_p1 = MancalaAI(depth=p1_depth, game_state=self)
        ai_p2 = MancalaAI(depth=p2_depth, game_state=self)

        for _ in range(games):
            self.reset_game()
            while self.winning_eval() is None:
                if self.current_player == 1:
                    move = (self.random_move_generator() if p1_type == 'random'
                            else ai_p1.best_move(self.clone(), p1_type == 'alpha-beta', True))
                else:
                    move = (self.random_move_generator() if p2_type == 'random'
                            else ai_p2.best_move(self.clone(), p2_type == 'alpha-beta', False))

                old_cur_player = self.current_player
                while old_cur_player == self.current_player:
                    self.play(move, old_cur_player, debug)
                    total_moves += 1

            winner = self.winning_eval()
            if winner == "Player 1":
                p1_wins += 1
            elif winner == "Player 2":
                p2_wins += 1
            else:
                ties += 1

            progress_bar.update(1)
        progress_bar.close()

        p1_string = 'Random' if p1_type == 'random' else ('AI - Minimax' if p1_type == 'minimax' else 'AI - Alpha-beta')
        p2_string = 'Random' if p2_type == 'random' else ('AI - Minimax' if p2_type == 'minimax' else 'AI - Alpha-beta')

        print(f'Player 1 ({p1_string}) Wins: {p1_wins}')
        print(f'Player 2 ({p2_string}) Wins: {p2_wins}')
        print(f'Ties: {ties}')
        print(f'Average number of moves per game: {total_moves/games}')



In [ ]:
class MancalaAI:
    def __init__(self, depth, game_state):
        self.depth = depth
        self.game_state = game_state

    def minimax(self, state, depth, maximizing_player, cur_player):
    # Termination conditions:
    # 1. Maximum search depth reached
    # 2. Game has ended (winning condition met)
        if depth == 0 or state.winning_eval() is not None:
          # Evaluate and return the current state's score
            return self.evaluate_state(state), None

        # Branch for maximizing player (Player 1)
        if maximizing_player:
          # Initialize with negative infinity to ensure any valid move is better
            value = -math.inf
            best_move = None

              # Explore all possible moves in Player 1's pits
            for i in range(state.p1_pits_index[0], state.p1_pits_index[1] + 1):
               # Only consider pits with stones
                if state.board[i] > 0:
                   # Create a clone of the current game state to simulate the move
                    new_state = state.clone()
                    # Simulate playing the move
                    new_state.play(i + 1, cur_player)

                    # Recursively explore the game tree
                # Switch to minimizing player for next level
                    new_score, _ = self.minimax(new_state, depth - 1, False, 3 - cur_player)
                    # Update best score and move if a better move is found
                    if new_score > value:
                        value = new_score
                        best_move = i + 1
            # Return the best score and corresponding move for maximizing player
            return value, best_move

            # Branch for minimizing player (typically Player 2)
        else:
          # Initialize with positive infinity to ensure any valid move is better
            value = math.inf
            best_move = None
            # Explore all possible moves in Player 2's pits
            for i in range(state.p2_pits_index[0], state.p2_pits_index[1] + 1):
              # Only consider pits with stones
                if state.board[i] > 0:
                  # Create a clone of the current game state to simulate the move
                    new_state = state.clone()
                    # Simulate playing the move
                    new_state.play(i + 1, cur_player)
                    # Recursively explore the game tree
                # Switch to maximizing player for next level
                    new_score, _ = self.minimax(new_state, depth - 1, True, 3 - cur_player)
                     # Update worst score and move if a better move is found
                    if new_score < value:
                        value = new_score
                        best_move = i + 1
                        # Return the best score and corresponding move for minimizing player
            return value, best_move

    def minimax_alpha_beta(self, state, depth, alpha, beta, maximizing_player, cur_player):
      # Termination conditions:
    # 1. Maximum search depth reached
    # 2. Game has ended (winning condition met)
        if depth == 0 or state.winning_eval() is not None:
           # Evaluate and return the current state's score
            return self.evaluate_state(state), None
        # Branch for maximizing player (Player 1)
        if maximizing_player:
          # Initialize with negative infinity to ensure any valid move is better
            value = -math.inf
            best_move = None
            # Explore all possible moves in Player 1's pits
            for i in range(state.p1_pits_index[0], state.p1_pits_index[1] + 1):
                # Only consider pits with stones
                if state.board[i] > 0:
                  # Create a clone of the current game state to simulate the move
                    new_state = state.clone()
                    # Simulate playing the move
                    new_state.play(i + 1, cur_player)
                     # Recursively explore the game tree
                # Switch to minimizing player for next level
                    new_score, _ = self.minimax_alpha_beta(new_state, depth - 1, alpha, beta, False, 3 - cur_player)
                    # Update best score and move if a better move is found
                    if new_score > value:
                        value = new_score
                        best_move = i + 1
                        # Update alpha (best score for maximizing player)
                    alpha = max(alpha, value)
                    # Alpha-Beta Pruning:
                # If beta is less than or equal to alpha,
                # further exploration won't change the outcome
                    if beta <= alpha:
                        break
            return value, best_move
        else:
            value = math.inf
            best_move = None
            for i in range(state.p2_pits_index[0], state.p2_pits_index[1] + 1):
                if state.board[i] > 0:
                    new_state = state.clone()
                    new_state.play(i + 1, cur_player)
                    new_score, _ = self.minimax_alpha_beta(new_state, depth - 1, alpha, beta, True, 3 - cur_player)
                    if new_score < value:
                        value = new_score
                        best_move = i + 1
                        # Update beta (best score for minimizing player)
                    beta = min(beta, value)
                    if beta <= alpha:
                        break
            return value, best_move

    def best_move(self, state, alpha_beta=False, first_player=False):
        """Determine the best move using either minimax or alpha-beta pruning."""
         # If Alpha-Beta pruning is selected, use minimax_alpha_beta method
        if alpha_beta:
           # Call minimax_alpha_beta with:
        # - current state
        # - predefined search depth
        # - initial alpha as negative infinity
        # - initial beta as positive infinity
        # - first_player flag to determine maximizing/minimizing player
        # - current player
        # Returns the best move (second element of the tuple)
            return self.minimax_alpha_beta(state, self.depth, -math.inf, math.inf, first_player, state.current_player)[1]
        else:
            # Call minimax with:
        # - current state
        # - predefined search depth
        # - first_player flag to determine maximizing/minimizing player
        # - current player
        # Returns the best move (second element of the tuple)
            return self.minimax(state, self.depth, first_player, state.current_player)[1]

    def evaluate_state(self, state):
        """Evaluate the state based on the difference between mancala scores."""
         # Calculate the score difference by subtracting Player 2's Mancala score
    # from Player 1's Mancala score
    # - Positive value indicates Player 1 is winning
    # - Negative value indicates Player 2 is winning
    # - Zero indicates a tied score
        return state.board[state.p1_mancala_index] - state.board[state.p2_mancala_index]



In [ ]:
def play_game_with_display(p1_type='random', p2_type='minimax', p1_depth=5, p2_depth=5, max_moves=50):
    """
    Play a Mancala game with visual display and configurable player types.

    Args:
    - p1_type: Player 1 strategy type ('random' or AI type)
    - p2_type: Player 2 strategy type ('random' or AI type)
    - p1_depth: Search depth for Player 1's AI (if applicable)
    - p2_depth: Search depth for Player 2's AI (if applicable)
    - max_moves: Maximum number of moves before game is considered a draw

    Displays game progress and final result
    """
    # Initialize the Mancala game
    game = Mancala()

    # Create AI players for both players with specified depths
    # Only used if non-random strategy is selected
    ai_p1 = MancalaAI(depth=p1_depth, game_state=game)
    ai_p2 = MancalaAI(depth=p2_depth, game_state=game)

    # Track total moves to prevent infinite game
    moves_count = 0

    # Game start announcement
    print(f"Game Start: Player 1 ({p1_type}) vs Player 2 ({p2_type})")
    # Display initial board state
    game.display_board()
    print("-" * 40)

    # Main game loop
    # Continues until:
    # 1. A winner is determined, or
    # 2. Maximum moves reached
    while game.winning_eval() is None and moves_count < max_moves:
        # Determine move for current player
        if game.current_player == 1:
            # If Player 1 is random, use random move generator
            # Otherwise, use AI to select best move
            move = (game.random_move_generator() if p1_type == 'random'
                    else ai_p1.best_move(game.clone(), p1_type == 'alpha-beta', True))
        else:
            # If Player 2 is random, use random move generator
            # Otherwise, use AI to select best move
            move = (game.random_move_generator() if p2_type == 'random'
                    else ai_p2.best_move(game.clone(), p2_type == 'alpha-beta', False))

        # Store current player before move (to handle multiple moves in same turn)
        old_cur_player = game.current_player
        # Announce the chosen move
        print(f"Player {old_cur_player} chooses pit: {move}")

        # Handle potential extra moves (if player lands in their own Mancala)
        while old_cur_player == game.current_player:
            # Play the move
            game.play(move, old_cur_player)
            # Increment move counter
            moves_count += 1

        # Display updated board state
        game.display_board()
        print("-" * 40)

    # Determine and announce game result
    winner = game.winning_eval()
    if winner:
        # If there's a winner, print the winner
        print(f"Game Over! Winner: {winner}")
    else:
        # If no winner (max moves reached), declare a draw
        print("Game ended in a draw")

# Example usage: Play a game with random moves for both players
print("Random vs Random")
play_game_with_display(p1_type='random', p2_type='random')

Random vs Random
Game Start: Player 1 (random) vs Player 2 (random)
P1               P2
     ____0____     
1 -> | 4 | 4 | <- 6
2 -> | 4 | 4 | <- 5
3 -> | 4 | 4 | <- 4
4 -> | 4 | 4 | <- 3
5 -> | 4 | 4 | <- 2
6 -> |_4_|_4_| <- 1
         0         
Turn: P1
----------------------------------------
Player 1 chooses pit: 1
P1               P2
     ____0____     
1 -> | 0 | 4 | <- 6
2 -> | 5 | 4 | <- 5
3 -> | 5 | 4 | <- 4
4 -> | 5 | 4 | <- 3
5 -> | 5 | 4 | <- 2
6 -> |_4_|_4_| <- 1
         0         
Turn: P2
----------------------------------------
Player 2 chooses pit: 11
P1               P2
     ____1____     
1 -> | 1 | 5 | <- 6
2 -> | 5 | 5 | <- 5
3 -> | 5 | 0 | <- 4
4 -> | 5 | 4 | <- 3
5 -> | 5 | 4 | <- 2
6 -> |_4_|_4_| <- 1
         0         
Turn: P1
----------------------------------------
Player 1 chooses pit: 1
P1               P2
     ____1____     
1 -> | 0 | 5 | <- 6
2 -> | 6 | 5 | <- 5
3 -> | 5 | 0 | <- 4
4 -> | 5 | 4 | <- 3
5 -> | 5 | 4 | <- 2
6 -> |_4_|_4_| <- 1
         

In [ ]:
print("Random vs Minimax")
play_game_with_display(p1_type='random', p2_type='minimax')

Random vs Minimax
Game Start: Player 1 (random) vs Player 2 (minimax)
P1               P2
     ____0____     
1 -> | 4 | 4 | <- 6
2 -> | 4 | 4 | <- 5
3 -> | 4 | 4 | <- 4
4 -> | 4 | 4 | <- 3
5 -> | 4 | 4 | <- 2
6 -> |_4_|_4_| <- 1
         0         
Turn: P1
----------------------------------------
Player 1 chooses pit: 2
P1               P2
     ____0____     
1 -> | 4 | 4 | <- 6
2 -> | 0 | 4 | <- 5
3 -> | 5 | 4 | <- 4
4 -> | 5 | 4 | <- 3
5 -> | 5 | 4 | <- 2
6 -> |_5_|_4_| <- 1
         0         
Turn: P2
----------------------------------------
Player 2 chooses pit: 10
P1               P2
     ____1____     
1 -> | 4 | 5 | <- 6
2 -> | 0 | 5 | <- 5
3 -> | 5 | 5 | <- 4
4 -> | 5 | 0 | <- 3
5 -> | 5 | 4 | <- 2
6 -> |_5_|_4_| <- 1
         0         
Turn: P1
----------------------------------------
Player 1 chooses pit: 1
P1               P2
     ____1____     
1 -> | 0 | 5 | <- 6
2 -> | 1 | 5 | <- 5
3 -> | 6 | 5 | <- 4
4 -> | 6 | 0 | <- 3
5 -> | 6 | 4 | <- 2
6 -> |_5_|_4_| <- 1
       

In [ ]:
print("Random vs Alpha-Beta")
play_game_with_display(p1_type='random', p2_type='alpha-beta')

Random vs Alpha-Beta
Game Start: Player 1 (random) vs Player 2 (alpha-beta)
P1               P2
     ____0____     
1 -> | 4 | 4 | <- 6
2 -> | 4 | 4 | <- 5
3 -> | 4 | 4 | <- 4
4 -> | 4 | 4 | <- 3
5 -> | 4 | 4 | <- 2
6 -> |_4_|_4_| <- 1
         0         
Turn: P1
----------------------------------------
Player 1 chooses pit: 2
P1               P2
     ____0____     
1 -> | 4 | 4 | <- 6
2 -> | 0 | 4 | <- 5
3 -> | 5 | 4 | <- 4
4 -> | 5 | 4 | <- 3
5 -> | 5 | 4 | <- 2
6 -> |_5_|_4_| <- 1
         0         
Turn: P2
----------------------------------------
Player 2 chooses pit: 10
P1               P2
     ____1____     
1 -> | 4 | 5 | <- 6
2 -> | 0 | 5 | <- 5
3 -> | 5 | 5 | <- 4
4 -> | 5 | 0 | <- 3
5 -> | 5 | 4 | <- 2
6 -> |_5_|_4_| <- 1
         0         
Turn: P1
----------------------------------------
Player 1 chooses pit: 4
P1               P2
     ____1____     
1 -> | 4 | 5 | <- 6
2 -> | 0 | 5 | <- 5
3 -> | 5 | 5 | <- 4
4 -> | 0 | 0 | <- 3
5 -> | 6 | 5 | <- 2
6 -> |_6_|_5_| <- 1
 

In [ ]:
game = Mancala()
game.match_analysis(p1_type='random', p2_type='random')
game.match_analysis(games=1, p1_type='random', p2_type='minimax')
game.match_analysis(p1_type='random', p2_type='minimax')
game.match_analysis(p1_type='random', p2_type='minimax', p2_depth=5)
game.match_analysis(p1_type='random', p2_type='alpha-beta')
game.match_analysis(p1_type='random', p2_type='alpha-beta', p2_depth=5)
# game.match_analysis(p1_type='random', p2_type='alpha-beta', p2_depth=10)



100%|██████████| 100/100 [00:00<00:00, 3497.73it/s]


Player 1 (Random) Wins: 43
Player 2 (Random) Wins: 52
Ties: 5
Average number of moves per game: 45.29


100%|██████████| 1/1 [00:00<00:00,  5.88it/s]


Player 1 (Random) Wins: 0
Player 2 (AI - Minimax) Wins: 1
Ties: 0
Average number of moves per game: 33.0


100%|██████████| 100/100 [00:21<00:00,  4.66it/s]


...

Player 1 (Random) Wins: 0
Player 2 (AI - Minimax) Wins: 100
Ties: 0
Average number of moves per game: 34.78


100%|██████████| 100/100 [00:05<00:00, 19.53it/s]


Player 1 (Random) Wins: 0
Player 2 (AI - Alpha-beta) Wins: 100
Ties: 0
Average number of moves per game: 34.83


100%|██████████| 100/100 [00:05<00:00, 17.34it/s]

Player 1 (Random) Wins: 0
Player 2 (AI - Alpha-beta) Wins: 100
Ties: 0
Average number of moves per game: 34.93
